# 线性回归的简洁实现
通过使用深度学习框架来简洁地实现线性回归模型生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 1.3349, -0.4891],
         [ 0.6737, -0.8463],
         [-0.8536,  1.0120],
         [ 0.3281, -1.6386],
         [ 0.9560, -0.2825],
         [-0.8876, -0.2120],
         [-1.2847, -0.0313],
         [ 1.4116,  1.4981],
         [-0.6351, -0.4749],
         [-1.1608,  0.0596]]),
 tensor([[ 8.5406],
         [ 8.4136],
         [-0.9637],
         [10.4335],
         [ 7.0753],
         [ 3.1395],
         [ 1.7486],
         [ 1.9311],
         [ 4.5390],
         [ 1.6888]])]

使用框架的预定义好的层

In [3]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

In [4]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是MESLoss类，也称为平方L2范数

In [5]:
loss = nn.MSELoss()

实例化SGD实例

In [6]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练过程代码与我们从零开始实现时所做的非常相似

In [7]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000248
epoch 2, loss 0.000104
epoch 3, loss 0.000105


比较生成数据集的真实参数和通过有线数据训练获得的模型参数

In [8]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0009, -0.0007])
b的估计误差： tensor([-2.8610e-05])
